# Stochastic Volume Calculations

OpenMC has a capability to stochastically determine volumes of cells, materials, and universes. The method works by overlaying a bounding box, sampling points from within the box, and seeing what fraction of points were found in a desired domain. The benefit of doing this stochastically (as opposed to equally-spaced points), is that it allows us to give an error estimate on each stochastic quantity.

In this example, we will use the stochastic volume capability in OpenMC to calculation microscopic cross sections on a heterogeneous cell.

In [1]:
from math import pi
import openmc

import sys
sys.path.append('../..')
from inputs import *

First let's start by creating our materials and geometry:

In [2]:
fuel = openmc.Material(1)
fuel.add_element('U', 1, enrichment=3.0)
fuel.add_element('O', 2)
fuel.set_density('g/cm3', 10.0)

water = openmc.Material(2)
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.add_s_alpha_beta('c_H_in_H2O')
water.set_density('g/cm3', 1)

mats = openmc.Materials([fuel, water])
mats.export_to_xml()

In [3]:
s1 = openmc.Sphere(R=10.0, boundary_type='vacuum')
s2 = openmc.Sphere(R=5.0)

inner_sphere = openmc.Cell(1, fill=fuel, region=-s2)
outer_sphere = openmc.Cell(2, fill=water, region=+s2 & -s1)
root = openmc.Universe(0, cells=(inner_sphere, outer_sphere))

geom = openmc.Geometry(root)
geom.export_to_xml()

The `VolumeCalculation` object accepts four arguments: the domains to find volumes of, how many samples you want, the lower-left coordinates of a bounding box, and the upper-right coordinates of a bounding box. For many cells, the bounding-box can be determined automatically so there is no need to specify the lower-left and upper-right coordinates in such cases.

In [4]:
lower_left, upper_right = geom.bounding_box
cell_vol = openmc.VolumeCalculation([inner_sphere, outer_sphere],
                                    1000000, lower_left, upper_right)
univ_vol = openmc.VolumeCalculation([root], 1000000, lower_left, upper_right)

After we've created the `VolumeCalculation` object, we need to assign it to the `Settings.volume_calculations` property.

In [5]:
settings = openmc.Settings()
settings.volume_calculations = [cell_vol, univ_vol]
settings.run_mode = 'volume'
settings.export_to_xml()

Finally, let's ask for some multi-group cross sections.

In [6]:
groups = openmc.mgxs.EnergyGroups((0.0, 0.625, 20.0e6))
totals = [openmc.mgxs.TotalXS(inner_sphere, groups=groups, by_nuclide=True),
          openmc.mgxs.TotalXS(root, groups=groups, by_nuclide=True)]

tallies = openmc.Tallies()
for mgxs in totals:
    tallies += mgxs.tallies.values()
tallies.export_to_xml()

In [7]:
openmc.calculate_volumes()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

0

Let's compare these estimates with the true values. Easy enough to calculate since we know that spheres are $\frac{4}{3} \pi r^3$.

In [8]:
v0 = 4/3*pi*s1.r**3
v1 = 4/3*pi*s2.r**3
v2 = v0 - v1 
print('Cell 1: {} cm^3'.format(v1))
print('Cell 2: {} cm^3'.format(v2))
print('Universe 0: {} cm^3'.format(v0))

Cell 1: 523.5987755982989 cm^3
Cell 2: 3665.191429188092 cm^3
Universe 0: 4188.790204786391 cm^3


## The `VolumeCalculation` object

We can also directly look at the results of a volume calculation using the HDF5 file that was output.

In [9]:
results = openmc.VolumeCalculation.from_hdf5('volume_1.h5')

In [10]:
results.volumes

{1: (524.38400000000001, 1.9799225794318323),
 2: (3662.5359999999996, 3.9857393352681756)}

In [11]:
results.atoms_dataframe

,Cell,Nuclide,Atoms,Uncertainty
0,1,U234,2.854560e+21,1.077799e+19
1,1,U235,3.552973e+23,1.341500e+21
2,1,U238,1.134003e+25,4.281669e+22
3,1,O16,2.338750e+25,8.830447e+22
4,1,O17,8.867225e+21,3.348008e+19
5,2,O16,1.224632e+26,1.332700e+23
6,2,H1,2.449263e+26,2.665401e+23


Note that the quantities here are total number of atoms rather than atom density. This is because in order to get density, you have to divide by the volume, which itself is a stochastic quantity. Thus, it's more accurate to report uncertainties on the total number of atoms.

## Microscopic Cross Sections

With an estimate of the volume and atom densities over a region, we are able to calculate microscopic multi-group cross sections over that region even if it is heterogeneous. Let's run in eigenvalue mode so that we can look at our microscopic multi-group cross sections.

In [12]:
settings.run_mode = 'eigenvalue'
settings.source = openmc.Source(space=openmc.stats.Point())
settings.batches = 50
settings.inactive = 0
settings.particles = 1000
settings.export_to_xml()

In [13]:
openmc.run()


                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #######################     %%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%

0

Now we'll load data from the statepoint into our MGXS objects. Note that when the `StatePoint` is instantiated, it will automatically look for `volume_*.h5` files and pull in the volume estimates.

In [14]:
sp = openmc.StatePoint('statepoint.50.h5')
for mgxs in totals:
    mgxs.load_from_statepoint(sp)

Now let's look at the microscopic MGXS for the inner sphere.

In [15]:
inner_sphere_total = totals[0]
inner_sphere_total.get_pandas_dataframe(xs_type='micro')

,cell,group in,nuclide,mean,std. dev.
5,1,1,U234,13.250421,0.863436
6,1,1,U235,11.222435,0.181040
7,1,1,U238,8.804746,0.127149
8,1,1,O16,3.170931,0.045797
9,1,1,O17,1.881418,0.026814
0,1,2,U234,76.556902,2.268418
1,1,2,U235,409.100862,12.070416
2,1,2,U238,10.950559,0.336415
3,1,2,O16,3.910513,0.121509
4,1,2,O17,3.941293,0.122062


And for the real test, let's see if we can get microscopic MGXS over our heterogeneous region (the entire universe).

In [16]:
univ_total = totals[1]
univ_total.get_pandas_dataframe(xs_type='micro')

,universe,group in,nuclide,mean,std. dev.
6,0,1,U234,41.493102,2.675590
7,0,1,U235,35.142553,0.460971
8,0,1,U238,27.571669,0.302485
9,0,1,O16,3.563967,0.021608
10,0,1,O17,5.891576,0.063161
11,0,1,H1,7.635150,0.041791
0,0,2,U234,28.030884,0.651385
1,0,2,U235,149.790008,3.456789
2,0,2,U238,4.009486,0.098689
3,0,2,O16,3.853123,0.066171


We see in this particular contrived example, the microscopic cross sections for U235 and U238 are quite different because, even though the reaction rates are the same, the average flux is much higher in the overall model than only in the inner sphere.